In [1]:
''' This is a sample for celestial navigation for a stationary observer 
    © August Linnman, 2025, email: august@linnman.net
    MIT License (see LICENSE file)

    Jupyter notebook for ordinary three-fix celestial navigation (stationary).
'''

# Importing Python libraries
from time import time
from starfix import Sight, SightCollection, get_representation, get_google_map_string,\
                    IntersectError, LatLonGeodetic, Circle


In [2]:
# SIGHTS

def get_starfixes (drp_pos : LatLonGeodetic) -> SightCollection :
    ''' Returns a list of used star fixes (SightCollection) '''

    a = Sight (   object_name          = "Sun",
                set_time             = "2024-05-05 15:55:18+00:00",
                gha_time_0           = "45:50.4",
                gha_time_1           = "60:50.4",
                decl_time_0          = "16:30.6",
                decl_time_1          = "16:31.3",
                measured_alt         = "55:7:54.4",
                estimated_position   = drp_pos
                )

    b = Sight (   object_name          = "Sun",
                set_time             = "2024-05-05 23:01:19+00:00",
                gha_time_0           = "165:50.8",
                gha_time_1           = "180:50.8",
                decl_time_0          = "16:36.2",
                decl_time_1          = "16:36.9",
                measured_alt         = "19:28:27.4",
                )

    c = Sight (   object_name          = "Vega",
                set_time             = "2024-05-06 04:04:13+00:00",
                gha_time_0           = "284:30.4",
                gha_time_1           = "299:32.9",
                decl_time_0          = "38:48.1",
                measured_alt         = "30:16:15.7",
                sha_diff             = "80:33.4",
                )
    return SightCollection ([a, b, c])


In [3]:
# SIGHT REDUCTION.

the_pos = LatLonGeodetic (90, -40) # Rough DRP position

intersections = None
collection = None
the_map = None
try:
    intersections, _, _, collection =\
            SightCollection.get_intersections_conv (return_geodetic=True,
                                                    estimated_position=the_pos,
                                                    get_starfixes=get_starfixes,
                                                    assume_good_estimated_position=True)
    
    assert intersections is not None
    assert collection is not None
    print (get_representation(intersections,1))
    assert isinstance (intersections, LatLonGeodetic)
    print ("Google Map Coordinate = " + get_google_map_string(intersections,4))

except IntersectError as ve:
    print ("Cannot perform a sight reduction. Bad sight data.\n" + str(ve))
    if ve.coll_object is not None:
        if isinstance (ve.coll_object, SightCollection):
            collection = ve.coll_object

if isinstance (intersections, tuple):
    intersections = None

if collection is not None:
    the_map = collection.render_folium (intersections)
the_map

(Geodetic) (N 41°,51.3′;W 87°,38.8′)
Google Map Coordinate = (Geodetic) 41.8552,-87.6464
